# Import

In [1]:
from data_utils import * 
from model_utils import *
from utils import FLClient, FLServer
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

/Users/gadmohamed/miniforge3/envs/env_tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

# Load CIFAR-10 for FL

In [2]:
dataset = 'cifar10'
num_classes = 10 if dataset == 'cifar10' else 100 
pub_num_classes = 100 if num_classes == 10 else 10
datadir = '../data'
partition = 'iid' 
n_parties = 5
beta = 0.5

(X_train, y_train, X_test, y_test, net_dataidx_map) = partition_data('cifar10', datadir=datadir, partition = partition, n_parties = n_parties, beta = beta)
(X_train_public, y_train_public, X_test_public, y_test_public, net_dataidx_map_public) = partition_data('cifar100', datadir=datadir, partition = 'iid', n_parties = 10, beta = 0.5)
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)
y_train_public_cat = to_categorical(y_train_public, num_classes=pub_num_classes)
y_test_public_cat = to_categorical(y_test_public, num_classes=pub_num_classes)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, y_train_cat.shape, y_test_cat.shape)
print(X_train_public.shape, y_train_public.shape, X_test_public.shape, y_test_public.shape, y_train_public_cat.shape, y_test_public_cat.shape)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,) (50000, 10) (10000, 10)
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,) (50000, 100) (10000, 100)


In [3]:
local_sets = [] 
test_sets = []
public_set = (X_train_public[net_dataidx_map_public[0]], y_train_public_cat[net_dataidx_map_public[0]])
for i in range(n_parties):
    local_sets.append((X_train[net_dataidx_map[i]], y_train_cat[net_dataidx_map[i]]))
    test_sets.append((X_test, y_test_cat))
    
print(public_set[0].shape, public_set[1].shape)
for i in range(n_parties):
    print('client ', i, ' ', local_sets[i][0].shape, local_sets[i][1].shape)
    print(test_sets[i][0].shape, test_sets[i][1].shape)
    print() 


(5000, 32, 32, 3) (5000, 100)
client  0   (10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)

client  1   (10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)

client  2   (10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)

client  3   (10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)

client  4   (10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)



In [4]:
aggregation_method = 'soft_labels'
aug = False
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1

fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
server = FLServer(fl_params)

In [5]:
sample = X_train[0] 
sample = np.expand_dims(sample, axis=0)
# to tensor 
t_sample = torch.from_numpy(sample).float()
server.clients[0].model(t_sample)

(tensor([[-0.5848,  0.1384,  0.7466,  1.3653, -1.4365,  1.2980,  0.0129, -0.7767,
          -0.5729,  0.6902]], grad_fn=<AddmmBackward0>),
 tensor([[0.0356, 0.0733, 0.1347, 0.2501, 0.0152, 0.2338, 0.0647, 0.0294, 0.0360,
          0.1273]], grad_fn=<SoftmaxBackward0>))

In [6]:
aggregation_method = 'compressed_soft_labels'
aug = False
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1

fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
compressed_server = FLServer(fl_params)

In [7]:
from utils import size_of 
soft_labels = server.clients[0].get_soft_labels()
compressed_soft_labels = compressed_server.clients[0].get_soft_labels(normalize = True, compress = True)
clientmodel = server.clients[0].model
ssize = size_of(soft_labels, size = 'KB')
cssize = size_of(compressed_soft_labels, size = 'KB')
clientmodel_state = clientmodel.state_dict()
msize = size_of(clientmodel_state, size = 'KB')

cssize, ssize, msize 


(50.0, 200.0, 3332.952)

## FedAvg

In [ ]:
aggregation_method = 'weights'
aug = False
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1

fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
server = FLServer(fl_params)

FL_acc = []
for t in range(fl_params['tot_T']):
    avg_acc, min_acc, max_acc, avg_train_acc = server.global_update()
    FL_acc.append(avg_acc)
    print(f"Round {t} accuracy: avg: {avg_acc} min: {min_acc}  max: {max_acc} avg_train: {avg_train_acc}")
print("Final accuracy: ", FL_acc[-1])
print() 

client_accs = []
for c, client in enumerate(server.clients):
    acc = client.local_benchmark()
    client_accs.append(acc)
    print(f"Client {c} local benchmark accuracy: {acc}")
print("Client accuracies: ", client_accs, "  ", np.mean(client_accs))
print()

server.save_assets()


## FedMD

In [ ]:
aggregation_method = 'soft_labels'
aug = False
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1


fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
server = FLServer(fl_params)

FL_acc = []
for t in range(fl_params['tot_T']):
    avg_acc, min_acc, max_acc, avg_train_acc = server.global_update()
    FL_acc.append(avg_acc)
    print(f"Round {t} accuracy: avg: {avg_acc} min: {min_acc}  max: {max_acc} avg_train: {avg_train_acc}")
print("Final accuracy: ", FL_acc[-1])
print() 

# client_accs = []
# for c, client in enumerate(server.clients):
#     acc = client.local_benchmark()
#     client_accs.append(acc)
#     print(f"Client {c} local benchmark accuracy: {acc}")
# print("Client accuracies: ", client_accs, "  ", np.mean(client_accs))
# print()

server.save_assets()


## CFedAKD

In [ ]:
aggregation_method = 'compressed_soft_labels'
aug = True
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1


fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
server = FLServer(fl_params)

FL_acc = []
for t in range(fl_params['tot_T']):
    avg_acc, min_acc, max_acc, avg_train_acc = server.global_update()
    FL_acc.append(avg_acc)
    print(f"Round {t} accuracy: avg: {avg_acc} min: {min_acc}  max: {max_acc} avg_train: {avg_train_acc}")
print("Final accuracy: ", FL_acc[-1])
print() 

# client_accs = []
# for c, client in enumerate(server.clients):
#     acc = client.local_benchmark()
#     client_accs.append(acc)
#     print(f"Client {c} local benchmark accuracy: {acc}")
# print("Client accuracies: ", client_accs, "  ", np.mean(client_accs))
# print()

server.save_assets()


## FedAKD

In [ ]:
aggregation_method = 'soft_labels'
aug = True
weighting = 'uniform'
private =   False
hyperparameter_tuning = False
C = 1


fl_params = {
    'client_num': len(local_sets),
    'tot_T': 50, 
    'C': C,
    'local_sets': local_sets,
    'test_sets': test_sets,
    'public_set': public_set,
    'batch_size': 32,
    'epochs': 3, 
    'lr': 0.005,
    'aggregate': aggregation_method, # 'grads', 'compressed_soft_labels', 'soft_labels'
    'hyperparameter_tuning': hyperparameter_tuning, 
    'weighting': weighting, # 'uniform', 'performance_based'
    'default_client_id': 1, 
    'augment': aug, 
    'private': private,
    'max_grad_norm': 1.0,
    'delta': 1e-4,
    'epsilon': 5,
    'local_benchmark_epochs': 70
}

N_pub = len(fl_params['public_set'][0])
exp_path = f"../fl_results/{dataset}/DP{private}/N_pub{N_pub}/Agg{fl_params['aggregate']}_C{fl_params['C']}_HT{fl_params['hyperparameter_tuning']}_Aug{fl_params['augment']}_W{fl_params['weighting']}"
fl_params['exp_path'] = exp_path
server = FLServer(fl_params)

FL_acc = []
for t in range(fl_params['tot_T']):
    avg_acc, min_acc, max_acc, avg_train_acc = server.global_update()
    FL_acc.append(avg_acc)
    print(f"Round {t} accuracy: avg: {avg_acc} min: {min_acc}  max: {max_acc} avg_train: {avg_train_acc}")
print("Final accuracy: ", FL_acc[-1])
print() 

# client_accs = []
# for c, client in enumerate(server.clients):
#     acc = client.local_benchmark()
#     client_accs.append(acc)
#     print(f"Client {c} local benchmark accuracy: {acc}")
# print("Client accuracies: ", client_accs, "  ", np.mean(client_accs))
# print()

server.save_assets()
